## Importing Libraries

In [18]:
import os
import cv2
import warnings
import numpy as np
from tkinter import *
import tensorflow as tf
from tkinter.ttk import *
from PIL import ImageTk,Image
from keras.models import Sequential
from keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout

## Dark spots Data Creation

In [19]:
features,target = [],[]

for x in ['nodarkspots', 'darkspots']:
    ImagesNamesList=os.listdir("images//train3" + "/" + str(x))
    for y in ImagesNamesList:
        Imgarr=cv2.imread(r"images//train3" + "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(50,50))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="nodarkspots":
                target.append(0)
            else:
                target.append(1)
    print("In Folder", x)


dark_spot_features = np.array(features)
dark_spot_target = np.array(target)

print(dark_spot_features.shape)
print(dark_spot_target.shape)

np.savez_compressed('datasets/dark_spot_data.npz', dark_spot_features, dark_spot_target)


In Folder nodarkspots
In Folder darkspots
(610, 50, 50, 3)
(610,)


## Puffy eyes Data Creation

In [20]:
features,target = [],[]

for x in ['no puffy eyes', 'puffy eyes']:
    ImagesNamesList=os.listdir("images//train2" + "/" + str(x))
    for y in ImagesNamesList:
        Imgarr=cv2.imread(r"images//train2" + "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(50,50))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="no puffy eyes":
                target.append(0)
            else:
                target.append(1)
    print("In Folder", x)


eyes_features = np.array(features)
eyes_target = np.array(target)

print(eyes_features.shape)
print(eyes_target.shape)

np.savez_compressed('datasets/eyes_data.npz', eyes_features, eyes_target)


In Folder no puffy eyes
In Folder puffy eyes
(607, 50, 50, 3)
(607,)


## Wrinkles Data Creation 

In [21]:
features,target = [],[]

for x in ['NoWrinkles', 'wrinkles']:
    ImagesNamesList=os.listdir("images//train1" + "/" + str(x))
    for y in ImagesNamesList:
        Imgarr=cv2.imread(r"images//train1" + "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(50,50))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="NoWrinkles":
                target.append(0)
            else:
                target.append(1)
    print("In Folder", x)


wrinkles_features = np.array(features)
wrinkles_target = np.array(target)

print(wrinkles_features.shape)
print(wrinkles_target.shape)

np.savez_compressed('datasets/wrinkles_data.npz', wrinkles_features, wrinkles_target)

In Folder NoWrinkles
In Folder wrinkles
(596, 50, 50, 3)
(596,)


In [22]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255
    return image

## Models Creation and  Compilation 

In [23]:
for a in os.listdir('datasets/'):
    data = np.load(r'datasets/'+str(a))
    features , target = data['arr_0'],data['arr_1']

    features_train,features_test,target_train,target_test=train_test_split(features,target,test_size=0.2)
    print(features_train.shape)
    print(target_train.shape)
    print(features_test.shape)
    print(target_test.shape)

    dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)
    batches=dataGen.flow(features_train,target_train,batch_size=20)
    images,labels=next(batches)

    target_train=to_categorical(target_train)

    model=Sequential()
    model.add(Conv2D(100,(3,3),activation="relu",input_shape=(50,50,3)))
    model.add(Conv2D(200,(3,3),activation="relu"))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(100,(3,3),activation="relu"))
    model.add(Conv2D(100,(3,3),activation="relu"))
    model.add(Conv2D(100,(3,3),activation="relu"))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dense(2,activation="softmax"))

    model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

    model.fit(dataGen.flow(features_train,target_train,batch_size=20),epochs=20)

    model_json=model.to_json()
    with open(str(a[:6])+".json","w") as abc:
        abc.write(model_json)
        abc.close
    model.save_weights("models/"+str(a[:6])+".h5")

(488, 50, 50, 3)
(488,)
(122, 50, 50, 3)
(122,)
Epoch 1/20
25/25 [==============================] - 20s 779ms/step - loss: 12.1152 - accuracy: 0.4836
Epoch 2/20
25/25 [==============================] - 20s 805ms/step - loss: 0.6909 - accuracy: 0.5512
Epoch 3/20
25/25 [==============================] - 23s 929ms/step - loss: 0.6380 - accuracy: 0.6148
Epoch 4/20
25/25 [==============================] - 23s 929ms/step - loss: 0.6335 - accuracy: 0.7111
Epoch 5/20
25/25 [==============================] - 22s 882ms/step - loss: 0.5513 - accuracy: 0.7152
Epoch 6/20
25/25 [==============================] - 21s 820ms/step - loss: 0.5447 - accuracy: 0.7234
Epoch 7/20
25/25 [==============================] - 20s 796ms/step - loss: 0.4431 - accuracy: 0.8033
Epoch 8/20
25/25 [==============================] - 20s 811ms/step - loss: 0.3970 - accuracy: 0.8094
Epoch 9/20
25/25 [==============================] - 20s 795ms/step - loss: 0.3347 - accuracy: 0.8791
Epoch 10/20
25/25 [=======================

## Model retriving

In [24]:
json_file=open(r"models/dark_s.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("models/dark_s.h5")

json_file=open(r"models/eyes_d.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model1=model_from_json(loaded_model_json)
loaded_model1.load_weights("models/eyes_d.h5")

json_file=open(r"models/wrinkl.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model2=model_from_json(loaded_model_json)
loaded_model2.load_weights("models/wrinkl.h5")

## Testing with an Image

In [25]:
Imgarr0 = cv2.imread(r"test_images/42.jpg")
Imgarr1=cv2.resize(Imgarr0,(50,50))
Imgarr = Imgarr1.reshape(-1, 50, 50, 3)

l = []

result = loaded_model.predict(Imgarr)
result = result[0]
if result[0] >= result[1]:
    l.append("dark spots")
else:
    l.append("no dark spots")

result = loaded_model1.predict(Imgarr)
result = result[0]
if result[0] >= result[1]:
    l.append("no puffy eyes")
else:
    l.append("puffy eyes")

result = loaded_model2.predict(Imgarr)
result = result[0]
if result[0] >= result[1]:
    l.append("no wrinkles on face")
else:
    l.append("wrinkles on face")
print(l)

['no dark spots', 'puffy eyes', 'wrinkles on face']


## GUI Image 

In [26]:
rootg = Tk()
rootg.resizable(False,False)
rootg.geometry('450x450')
rootg.title('Facial Predictions')
u2 = Image.open("images/images_2.jpg")
u2 = u2.resize((450,450),Image.ANTIALIAS)
u2 = ImageTk.PhotoImage(u2)
Label(rootg,image=u2).place(x=0, y=0, width=450, height=450)
img = cv2.resize(Imgarr0, (300, 250))
im = Image.fromarray(img)
imgtk = ImageTk.PhotoImage(image=im) 
Label(rootg, image=imgtk).place(x=70,y=50)

Label(rootg, text = "Predictions are : "+str(l)).place(x=50,y=350)
rootg.mainloop()